In [ ]:
# Imports
import os 
import pandas as pd
from django.core.exceptions import ValidationError
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
from datetime import date

In [ ]:
##Maternal Window
from edc_appointment.models import Appointment

subject_identifier = 'B142-040990037-9'

ons = OnScheduleCohortBSecQuart.objects.get(subject_identifier=subject_identifier)

sn = ons.schedule_name
ons.schedule_name = ons.schedule_name + 'x'
ons.save()

apps = Appointment.objects.filter(subject_identifier=subject_identifier,
                                  schedule_name=sn)

for ap0 in apps:

    ap0.schedule_name = ons.schedule_name
    ap0.save()

# resave enroll visit

cpe = CaregiverPreviouslyEnrolled.objects.get(subject_identifier=subject_identifier, visit_code='2000M')

cpe.save()

ap1 = Appointment.objects.get(subject_identifier=subject_identifier,
                              visit_code='2000M', schedule_name=sn)
ap0 = Appointment.objects.get(subject_identifier=subject_identifier,
                              visit_code='2000M', schedule_name=ons.schedule_name)

ap1.user_modified = ap0.user_modified
ap1.appt_status = ap0.appt_status
ap1.appt_reason = ap0.appt_reason
ap1.appt_datetime = ap0.appt_datetime

ap1.save()

try:
    v1 = MaternalVisit.objects.get(appointment=ap0)
    v1.appointment = ap1
    v1.save()
except:
    pass

apps.delete()
ons.delete()